# Create a route from Chicago to Columbus
This notebook will use the Line data that is stored in the database rather than use the NARN geojson file.


In [1]:
import geopandas as gpd
import networkx as nx
from shapely import geometry, ops
import numpy as np
import matplotlib.pyplot as plt
import requests
import pandas as pd
import folium
import time
from requests.auth import AuthBase
import os
from scipy.spatial import distance

In [2]:
URL ="http://score-web-1:8000"
URL1 = URL+"/api-token-auth/"
payload = {'username':'locomotives', 'password':'locomotives'}
URL2 = URL+"/api/line/add/"
URL3 = URL+'/api/railroad/'
URL4 = URL + "/api/line/"
URL5 = URL + "/api/route/add/"
URL6 = URL + "/api/yard/all/"
URL7 = URL + "/api/yard/"
URL8 = URL + "/api/lines/railroad/"

In [3]:
t = requests.post(URL1, data=payload )
token = t.json().get('token')

In [4]:
class TokenAuth(AuthBase):
    """ Implements a custom authentication scheme. """

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """ Attach an API token to a custom auth header. """
        r.headers['Authorization'] = "Token " + f'{self.token}'
        return r

In [5]:
r = requests.get(URL3, auth=TokenAuth(token))
railroads=r.json()['results']
railroads

[{'id': 1, 'code': 'BNSF', 'name': 'Burlington Northern and Santa Fe'},
 {'id': 2, 'code': 'CN', 'name': 'Canadian National Railway'},
 {'id': 3, 'code': 'CP', 'name': 'Canadian Pacific Railway'},
 {'id': 4, 'code': 'CSXT', 'name': 'CSX Transportation'},
 {'id': 5, 'code': 'NS', 'name': 'Norfolk Southern Railway'},
 {'id': 6, 'code': 'KCS', 'name': 'Kansas City Southern Railway'},
 {'id': 7, 'code': 'UP', 'name': 'Union Pacific'}]

In [6]:
response = requests.get(URL6, auth=TokenAuth(token))
yards = response.json()['results']
yards

[{'id': 1, 'name': 'Landers Yard'}, {'id': 2, 'name': 'Rickenbacker'}]

In [7]:
response = requests.get(URL7+'1', auth=TokenAuth(token))
start = response.json()['results']
start

{'id': 1,
 'code': 'LND',
 'name': 'Landers Yard',
 'city': 'Chicago',
 'state': 'IL',
 'location': 414008,
 'owner': 7}

In [8]:
response = requests.get(URL7+'2', auth=TokenAuth(token))
end = response.json()['results']
end

{'id': 2,
 'code': 'RIC',
 'name': 'Rickenbacker',
 'city': 'Columus',
 'state': 'OH',
 'location': 442152,
 'owner': 5}

In [9]:
def get_alt(lat, lng):
    st = ""
    # print(lng)
    for i in range(len(lng)):
        st+=str(lat[i])+','+str(lng[i])+'|'
    st=st[:-1]
    url = "https://maps.googleapis.com/maps/api/elevation/json?locations=" + st + "&key=" + API_KEY
    # print(url)
    response = requests.get(url)
    # result = response.json()['results']['elevation']
    # print(response.status)
    # print(response.json()['results'])
    results = response.json()['results']
    res = []
    for rec in results:
        res.append(rec['elevation'])
    # should put in a test for response status before returning an actual value
    return res

In [15]:
response = requests.get(URL8+'NS', auth=TokenAuth(token))

In [16]:
lines = response.json()['results']

In [17]:
response.status_code

200

In [18]:
len(lines)

34050

This is a test to see if the length provided in the NARN database is accurate. Unfortunately it is not - ugh. It seems to be off by as much 20% from this one case. As an quick check, we can determine the straight line distance between the start adn end and even it is greater than the provided length. The result is, we need to "fix" the length of the segments in the databse so that our shortest path algorithm is more accurate, though it may not make too much difference when it comes to determine the shortest route, just the resulting length will be in error.

In [19]:
np.array(lines[2]['distance']).sum()

656.5716965456393

In [20]:
lines[2]['length']

0.55419809

In [21]:
xy = np.reshape(np.array(lines[2]['xy']),(-1,2))

In [22]:
xy

array([[-9255937.92534617,  3784295.22753769],
       [-9255951.31151493,  3784365.25086021],
       [-9255949.98692431,  3784398.10422973],
       [-9255942.04071642,  3784426.26444515],
       [-9255928.79648001,  3784457.55356336],
       [-9255883.76685542,  3784520.13204488],
       [-9255863.9007791 ,  3784552.98583907],
       [-9255853.3056126 ,  3784587.40429019],
       [-9255849.33250866,  3784624.95185198],
       [-9255853.3056126 ,  3784660.93495479],
       [-9255858.60330717,  3784687.5310763 ],
       [-9255875.82053621,  3784721.95004541],
       [-9255991.04344495,  3784883.09373564]])

In [23]:
p1=xy[0]
p2=xy[-1]
np.linalg.norm(p2-p1)

590.2611279024937